In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import plotly.express as px
import random
import os
has_mps = torch.backends.mps.is_built()
# device = "cpu"
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [2]:
def set_seed(seed):
    random.seed(seed)                       # Python RNG
    np.random.seed(seed)                    # NumPy RNG
    torch.manual_seed(seed)                 # PyTorch CPU RNG
    torch.cuda.manual_seed(seed)            # PyTorch current GPU RNG
    torch.cuda.manual_seed_all(seed)        # PyTorch all GPU RNGs

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # Better reproducibility

    os.environ["PYTHONHASHSEED"] = str(seed)  # For hash-based functions

# Example usage
set_seed(18)

In [3]:
fn = "./datasets/CALCE/CALCE.csv"
df = pd.read_csv(fn)
df.columns

Index(['cycle', 'capacity_CS2_35', 'capacity_CS2_36', 'capacity_CS2_37',
       'capacity_CS2_38'],
      dtype='object')

In [4]:
spots_train_1 = df["capacity_CS2_36"].to_numpy().reshape(-1, 1)
spots_train_2 = df["capacity_CS2_37"].to_numpy().reshape(-1, 1)
spots_train_3 = df["capacity_CS2_38"].to_numpy().reshape(-1, 1)

spots_test = df["capacity_CS2_35"].to_numpy().reshape(-1, 1)

In [5]:
scaler = StandardScaler()
spots_train_1 = scaler.fit_transform(spots_train_1).flatten().tolist()
spots_train_2 = scaler.fit_transform(spots_train_2).flatten().tolist()
spots_train_3 = scaler.fit_transform(spots_train_3).flatten().tolist()

spots_test = scaler.transform(spots_test).flatten().tolist()

# Sequence Data Preparation
SEQUENCE_SIZE = 30

In [6]:


def to_sequences(seq_size, obs):
    x = []
    y = []
    for i in range(len(obs) - seq_size):
        window = obs[i:(i + seq_size)]
        after_window = obs[i + seq_size]
        x.append(window)
        y.append(after_window)
    return torch.tensor(x, dtype=torch.float32).view(-1, seq_size, 1), torch.tensor(y, dtype=torch.float32).view(-1, 1)

x_train_1, y_train_1 = to_sequences(SEQUENCE_SIZE, spots_train_1)
x_train_2, y_train_2 = to_sequences(SEQUENCE_SIZE, spots_train_2)
x_train_3, y_train_3 = to_sequences(SEQUENCE_SIZE, spots_train_3)
x_test, y_test = to_sequences(SEQUENCE_SIZE, spots_test)

# Setup data loaders for batch
'''TensorDataset
- TensorDataset is a special PyTorch wrapper that combines muliple tensors into a single dataset
- x_train and y_train must have the same first dimension
- this allows easy retrieval of (input, target) pairs during training
- makes integration with DataLoader seamless
'''

'''DataLoader
- Batches data for efficienct training
- Shuffles data befoe each epoch to prevent model from memorizing the order
It randomly select batch(32) size from train_dataset and group them into a batch [32, (sequence)10, 1] for x_train and [32, 1] for y_train
return as PyTorch Tensor for training
'''



'DataLoader\n- Batches data for efficienct training\n- Shuffles data befoe each epoch to prevent model from memorizing the order\nIt randomly select batch(32) size from train_dataset and group them into a batch [32, (sequence)10, 1] for x_train and [32, 1] for y_train\nreturn as PyTorch Tensor for training\n'

In [7]:
x_train_1.shape


torch.Size([852, 30, 1])

In [8]:
x_train_2.shape

torch.Size([852, 30, 1])

In [9]:
x_train = torch.cat((x_train_1, x_train_2, x_train_3), dim=0)
y_train = torch.cat((y_train_1, y_train_2, y_train_3), dim=0)

In [10]:
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
import torch
import torch.nn as nn
import numpy as np
import math

# Rotary Positional Encoding (RoPE)
class RotaryPositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(RotaryPositionalEncoding, self).__init__()
        self.d_model = d_model
        self.max_len = max_len
        
        # Initialize frequency for rotation
        self.freqs = 1.0 / (10000 ** (torch.arange(0, d_model, 2).float() / d_model))
        
    def _rotate_half(self, x):
        x1, x2 = x[..., :x.shape[-1]//2], x[..., x.shape[-1]//2:]
        return torch.cat((-x2, x1), dim=-1)
    
    def forward(self, q, k, positions):
        # q, k: [batch_size, nhead, seq_len, head_dim]
        # positions: [seq_len]
        
        # Expand positions to match batch size and number of heads
        # [seq_len] -> [seq_len, dim//2]
        seq_len = positions.shape[0]
        cos_pos = positions.unsqueeze(1) * self.freqs.to(positions.device)
        sin_pos = positions.unsqueeze(1) * self.freqs.to(positions.device)
        
        # [seq_len, dim//2] -> [seq_len, dim]
        cos_pos = torch.cat([cos_pos, cos_pos], dim=1)
        sin_pos = torch.cat([sin_pos, sin_pos], dim=1)
        
        # [seq_len, dim] -> [1, 1, seq_len, dim]
        cos_pos = cos_pos.unsqueeze(0).unsqueeze(0)
        sin_pos = sin_pos.unsqueeze(0).unsqueeze(0)
        
        # Apply rotary embeddings
        q_cos = cos_pos * q
        q_sin = sin_pos * self._rotate_half(q)
        q_out = q_cos + q_sin
        
        k_cos = cos_pos * k
        k_sin = sin_pos * self._rotate_half(k)
        k_out = k_cos + k_sin
        
        return q_out, k_out

# Modified TransformerEncoderLayer with RoPE
class RoPETransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1):
        super(RoPETransformerEncoderLayer, self).__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.head_dim = d_model // nhead
        
        # Multi-head attention components
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)
        self.o_proj = nn.Linear(d_model, d_model)
        
        # RoPE
        self.rope = RotaryPositionalEncoding(self.head_dim)
        
        # Feed forward network
        self.linear1 = nn.Linear(d_model, d_model * 4)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_model * 4, d_model)
        
        # Layer normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        
    def forward(self, src, mask=None):
        # src shape: [seq_len, batch_size, d_model]
        seq_len, batch_size, _ = src.shape
        
        # Apply first normalization
        src2 = self.norm1(src)
        
        # Project queries, keys, values
        q = self.q_proj(src2).view(seq_len, batch_size, self.nhead, self.head_dim)
        k = self.k_proj(src2).view(seq_len, batch_size, self.nhead, self.head_dim)
        v = self.v_proj(src2).view(seq_len, batch_size, self.nhead, self.head_dim)
        
        # Transpose for attention
        q = q.permute(1, 2, 0, 3)  # [batch_size, nhead, seq_len, head_dim]
        k = k.permute(1, 2, 0, 3)  # [batch_size, nhead, seq_len, head_dim]
        v = v.permute(1, 2, 0, 3)  # [batch_size, nhead, seq_len, head_dim]
        
        # Create position indices
        positions = torch.arange(seq_len, device=src.device)
        
        # Apply RoPE to queries and keys
        q_rope, k_rope = self.rope(q, k, positions)
        
        # Compute attention scores
        attn_output_weights = torch.matmul(q_rope, k_rope.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        # Apply causal mask if provided
        if mask is not None:
            attn_output_weights = attn_output_weights + mask.unsqueeze(0).unsqueeze(0)
        
        # Apply softmax to get attention weights
        attn_output_weights = torch.softmax(attn_output_weights, dim=-1)
        
        # Apply attention to values
        attn_output = torch.matmul(attn_output_weights, v)  # [batch_size, nhead, seq_len, head_dim]
        
        # Reshape and project output
        attn_output = attn_output.permute(2, 0, 1, 3).contiguous().view(seq_len, batch_size, self.d_model)
        attn_output = self.o_proj(attn_output)
        
        # Apply residual connection and dropout
        src = src + self.dropout1(attn_output)
        
        # Apply second normalization
        src2 = self.norm2(src)
        
        # Feed forward network
        src2 = self.linear2(self.dropout(torch.relu(self.linear1(src2))))
        
        # Apply residual connection and dropout
        src = src + self.dropout2(src2)
        
        return src

# Custom TransformerEncoder using RoPE
class RoPETransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers):
        super(RoPETransformerEncoder, self).__init__()
        self.layers = nn.ModuleList([encoder_layer for _ in range(num_layers)])
        
    def forward(self, src, mask=None):
        output = src
        for layer in self.layers:
            output = layer(output, mask=mask)
        return output

# Helper function to generate causal mask
def generate_causal_mask(sz, device):
    mask = torch.triu(torch.ones(sz, sz, device=device) * float('-inf'), diagonal=1)
    return mask

# Updated Transformer Model with RoPE
class TransformerModel(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
        super(TransformerModel, self).__init__()

        self.encoder = nn.Linear(input_dim, d_model)
        
        # Create RoPE transformer encoder layers
        encoder_layer = RoPETransformerEncoderLayer(d_model, nhead, dropout=dropout)
        self.transformer_encoder = RoPETransformerEncoder(encoder_layer, num_layers)
        
        self.decoder = nn.Linear(d_model, 1)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

    def forward(self, x):
        # x shape: [batch_size, seq_len, input_dim]
        batch_size, seq_len, _ = x.size()

        x = self.encoder(x)  # [batch_size, seq_len, d_model]
        x = x.transpose(0, 1)  # [seq_len, batch_size, d_model]
        
        # No separate positional encoding - RoPE is applied within the attention mechanism
        causal_mask = generate_causal_mask(seq_len, x.device)  # [seq_len, seq_len]
        
        x = self.transformer_encoder(x, mask=causal_mask)

        x = x.transpose(0, 1)  # [batch_size, seq_len, d_model]
        out = self.decoder(x[:, -1, :])  # take the last time step
        return out

In [12]:
model = TransformerModel().to(device)

# Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

epochs = 1000
early_stop_count = 0
min_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    train_losses = []
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        train_losses.append(loss.item())
        loss.backward() # calculate gradients
        optimizer.step() # update weights based on gradients and learning rate

    train_loss = np.mean(train_losses)

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step(val_loss)

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 5:
        print("Early stopping!")
        break
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    
# Evaluation
from sklearn.metrics import mean_absolute_error
# Evaluation
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch in test_loader:
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)
        outputs = model(x_batch)
        predictions.extend(outputs.squeeze().tolist())
        actuals.extend(y_batch.squeeze().tolist())



c:\Users\Mike\miniconda3\envs\py310\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Mike\miniconda3\envs\py310\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/1000, Train Loss: 0.1846, Validation Loss: 0.1616
Epoch 2/1000, Train Loss: 0.0211, Validation Loss: 0.1224
Epoch 3/1000, Train Loss: 0.0181, Validation Loss: 0.0908
Epoch 4/1000, Train Loss: 0.0184, Validation Loss: 0.0988
Epoch 5/1000, Train Loss: 0.0162, Validation Loss: 0.1146
Epoch 6/1000, Train Loss: 0.0157, Validation Loss: 0.0861
Epoch 7/1000, Train Loss: 0.0169, Validation Loss: 0.1412
Epoch 8/1000, Train Loss: 0.0154, Validation Loss: 0.1286
Epoch 9/1000, Train Loss: 0.0149, Validation Loss: 0.0542
Epoch 10/1000, Train Loss: 0.0151, Validation Loss: 0.0683
Epoch 11/1000, Train Loss: 0.0159, Validation Loss: 0.0942
Epoch 12/1000, Train Loss: 0.0152, Validation Loss: 0.1045
Epoch 13/1000, Train Loss: 0.0139, Validation Loss: 0.0734
Early stopping!


In [13]:
# Convert to numpy arrays and inverse transform
predictions = np.array(predictions).reshape(-1, 1)
actuals = np.array(actuals).reshape(-1, 1)

predictions_inv = scaler.inverse_transform(predictions)
actuals_inv = scaler.inverse_transform(actuals)

# Compute Scores
rmse = np.sqrt(np.mean((predictions_inv - actuals_inv) ** 2))
mae = mean_absolute_error(actuals_inv, predictions_inv)
re = np.mean(np.abs((actuals_inv - predictions_inv) / actuals_inv))  # Percentage form


print(f"Score (Relative Error): {re:.4f}")
print(f"Score (MAE): {mae:.4f}")
print(f"Score (RMSE): {rmse:.4f}")


Score (Relative Error): 0.0328
Score (MAE): 0.0148
Score (RMSE): 0.0321


In [14]:
# auto_regressive both improve performance and needs less training resources